In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.prompts import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
chat = ChatOpenAI(temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])


In [ ]:
# b = chat.predict("How many planets are there?")
# b

template = PromptTemplate.from_template("{country_a}와 {country_b}사이의 거리는 어떻게되나요?")
prompt = template.format(country_a="맥시코", country_b="인도")
chat.predict(prompt)

In [15]:
template = ChatPromptTemplate.from_messages([
    ("system", "당신은 지리 전문가입니다. 그리고 당신은 {language}로만 대답을 합니다."),
    ("ai", "안녕하세요 제 이름은 {name}입니다."),
    ("human", "{country_a}와 {country_b}사이의 거리는 어떻게되나요? 그리고 당신의 이름은 무었입니까?")
])
prompt = template.format_messages(language="일본어", name="길동", country_a="일본", country_b="한국")
chat.predict_messages(prompt)

AIMessage(content='안녕하세요! 일본과 한국 사이의 거리는 대략 900km 정도입니다. 제 이름은 길동입니다.')

In [8]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))
    
    
p = CommaOutputParser()
p.parse(" a ,  b ,  c ,   d")

['a', 'b', 'c', 'd']

In [16]:
template = ChatPromptTemplate.from_messages([
    ("system", "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items} in lowercase. Do Not reply with anything else."),
    ("human", "{question}")
])
# prompt = template.format_messages(
#     max_items=10,
#     question="What are the planets"
# )
prompt = template.format_messages(
    max_items=10,
    question="What are the colors"
)
result = chat.predict_messages(prompt)
p = CommaOutputParser()
p.parse(result.content)


['red',
 'orange',
 'yellow',
 'green',
 'blue',
 'indigo',
 'violet',
 'black',
 'white',
 'gray']

In [17]:
chain = template | chat | CommaOutputParser()
chain.invoke({
    "max_items": 5,
    "question":"What are the pokemons?"
})

['pikachu', 'charizard', 'bulbasaur', 'squirtle', 'jigglypuff']

In [3]:
chef_prompt  = ChatPromptTemplate.from_messages(
    [
        ("system","You are a world-class international chef. You create easy to follow recipies for any type of cuisins with easy to find ingredients."),
        ("human","I want to cook {cuisine} food"),
    ]
)
chef_chain = chef_prompt | chat
# chef_chain.invoke({
#     "cuisine":"korean"
# })

In [4]:
veg_chef_prompt = ChatPromptTemplate.from_messages([
    ("system","You are a vegetarian chef specialized on making traditional recipies vegetarian. You find a alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it."),
    ("human", "{recipe}"),
])

veg_chain = veg_chef_prompt | chat



In [5]:
final_chain = {"recipe": chef_chain} | veg_chain
final_chain.invoke({
    "cuisine":"korean"
})

Great choice! Korean cuisine is known for its bold flavors and unique combinations. Here's a recipe for a classic Korean dish called Bibimbap:

Ingredients:
- 2 cups cooked short-grain rice
- 1 cup thinly sliced beef (you can use sirloin or ribeye)
- 1 cup julienned carrots
- 1 cup julienned cucumber
- 1 cup bean sprouts
- 1 cup spinach
- 4 eggs
- 4 tablespoons gochujang (Korean chili paste)
- 2 tablespoons soy sauce
- 2 tablespoons sesame oil
- 2 cloves garlic, minced
- Salt and pepper to taste
- Vegetable oil for cooking

Instructions:
1. Marinate the beef: In a bowl, combine the beef, soy sauce, minced garlic, sesame oil, salt, and pepper. Mix well and let it marinate for at least 30 minutes.

2. Prepare the vegetables: Blanch the bean sprouts and spinach in boiling water for about 1 minute. Drain and rinse with cold water. Squeeze out excess water from the spinach and season with a pinch of salt. Set aside. Saute the carrots in a pan with a little vegetable oil until slightly softe

AIMessageChunk(content='For this Bibimbap recipe, here are some alternative ingredients and their preparation:\n\n1. Beef: Replace the beef with a plant-based meat substitute such as tempeh or seitan. Marinate the tempeh or seitan in the same marinade as the beef, using soy sauce, minced garlic, sesame oil, salt, and pepper. Let it marinate for at least 30 minutes before cooking. Sauté the marinated tempeh or seitan in a pan until browned and cooked through.\n\n2. Eggs: If you prefer to make this recipe vegan, you can replace the eggs with tofu scramble. Crumble firm tofu and season it with salt, pepper, and turmeric for color. Sauté the seasoned tofu in a pan until heated through.\n\n3. Gochujang (Korean chili paste): Look for a vegan-friendly gochujang that does not contain any animal-derived ingredients. Many brands offer vegan versions, but always check the label to be sure.\n\n4. Vegetable oil: Use any neutral-flavored oil such as canola or sunflower oil instead of vegetable oil.\

In [2]:
# 4.1 FewShotPromptTemplate

# t = PromptTemplate.from_template("What is the capital of {country}")
# t.format(country="Korea")

# t=PromptTemplate(template="What is the capital of {country}", input_variables=["country"])
# t.format(country="japan")

examples = [
{
"question": "What do you know about France?",
"answer": """
Here is what I know:
Capital: Paris
Language: French
Food: Wine and Cheese
Currency: Euro
""",
},
{
"question": "What do you know about Italy?",
"answer": """
I know this:
Capital: Rome
Language: Italian
Food: Pizza and Pasta
Currency: Euro
""",
},
{
"question": "What do you know about Greece?",
"answer": """
I know this:
Capital: Athens
Language: Greek
Food: Souvlaki and Feta Cheese
Currency: Euro
""",
},
]

example_template = """
    Human: {question}
    AI: {answer}
"""

example_prompt = PromptTemplate.from_template(example_template)
prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"]
)

chain = prompt | chat
chain.invoke({
    "country": "Germany"
})

'\n    Human: What do you know about France?\n    AI: \nHere is what I know:\nCapital: Paris\nLanguage: French\nFood: Wine and Cheese\nCurrency: Euro\n\n\n\n\n    Human: What do you know about Italy?\n    AI: \nI know this:\nCapital: Rome\nLanguage: Italian\nFood: Pizza and Pasta\nCurrency: Euro\n\n\n\n\n    Human: What do you know about Greece?\n    AI: \nI know this:\nCapital: Athens\nLanguage: Greek\nFood: Souvlaki and Feta Cheese\nCurrency: Euro\n\n\n\nHuman: What do you know about Germany?'